<a href="https://colab.research.google.com/github/jpiche0720/DS-Unit-2-Applied-Modeling/blob/master/module1-define-ml-problems/LS_DS_231_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [0]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',40)
# Path1 = energy consumption for electricity by state(target)
# path1half = path one btus
# Path2 = GDP of sector by state 2001-2017
# Path3 = SAAC Arts GDP 2001-2016
# Path4 = population estimate 2000-2010
# Path5 = population estimate 2010-2018

In [0]:
# Energy
path1 = 'annual_consumption_state.xls'

# GDP
path2 = 'gdp_all_areas_2001_to_2018.csv'

# Population 2000-2010
path4 ='annual_population_2000_to_2010.xls'
path4_cols = ['state','census','esimate_base','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010']

# Population 2010-2019
path5 = 'annual_population_2010_to_2017.xlsx'
path5_cols = ['state','census','esimate_base','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']


energy_data = pd.read_excel(path1,skiprows=1)# NA values - (D)

gdp_sector_data = pd.read_csv(path2,encoding='latin-1',low_memory=False,na_values='(NA)')

population0 = pd.read_excel(path4,skiprows=8,names=path4_cols,skipfooter=8)

population1 = pd.read_excel(path5,skiprows=8,names=path5_cols,skipfooter=3)

# Read in Population XL table

In [0]:
print(population0.shape)
population0.head()

(53, 14)


,state,census,esimate_base,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,.Alabama,4447207.0,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0,4779736.0,4785298.0
1,.Alaska,626933.0,627963.0,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0,710231.0,713985.0
2,.Arizona,5130247.0,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,5839077.0,6029141.0,6167681.0,6280362.0,6343154.0,6392017.0,6413737.0
3,.Arkansas,2673293.0,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,2781097.0,2821761.0,2848650.0,2874554.0,2896843.0,2915918.0,2921606.0
4,.California,33871653.0,33987977.0,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0,37253956.0,37349363.0


In [0]:
print(population1.shape)
population1.head()

(55, 13)


,state,census,esimate_base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,.Alabama,4779736.0,4780125.0,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0
1,.Alaska,710231.0,710249.0,713910.0,722128.0,730443.0,737068.0,736283.0,737498.0,741456.0,739700.0,735139.0,731545.0
2,.Arizona,6392017.0,6392288.0,6407172.0,6472643.0,6554978.0,6632764.0,6730413.0,6829676.0,6941072.0,7044008.0,7158024.0,7278717.0
3,.Arkansas,2915918.0,2916031.0,2921964.0,2940667.0,2952164.0,2959400.0,2967392.0,2978048.0,2989918.0,3001345.0,3009733.0,3017804.0
4,.California,37253956.0,37254519.0,37319502.0,37638369.0,37948800.0,38260787.0,38596972.0,38918045.0,39167117.0,39358497.0,39461588.0,39512223.0


In [0]:
# Drop na values for misc rows
population0 = population0.dropna()

population1 = population1.dropna()

# Reset index and drop index column created
population0 = population0.reset_index()
population0.columns.drop(['index'])

population1 = population1.reset_index()
population1.columns.drop(['index'])

Index(['state', 'census', 'esimate_base', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

In [0]:
# limited PR data
population0 = population0.drop(51)
population1 = population1.drop(51)
# limited Hawaii data
population0 = population0.drop(11)
population1 = population1.drop(11)

# Begin with State names, Year of Observation


### State name:

In [0]:
# Get a convienient list of State Names in ALphabetical Order
states = population1['state'].to_list()

In [0]:
# Format Strings of State Names
state_li = []
for i in range(len(states)):
    state_li.append(states[i].replace(".",""))

state_li

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [0]:
# Build a Year of observation for each state name 
yearsss = []
year_li = []
for i in range(2001,2018):
    yr = [i]*50
    yearsss.append(yr)
for sublist in yearsss:
    for item in sublist:
        year_li.append(item)


# Check Size for observations and state names
print('Year Length:',len(year_li))
print('State Length:',len(state_li*17))

Year Length: 850
State Length: 850


In [0]:
# Create Pandas Series and stick them together into one series for Observation identification
states = pd.Series(state_li*17)
years = pd.Series(year_li)
data = states+years.astype(str)
print('Data Length:',len(data))
data.head()

Data Length: 850


0       Alabama2001
1        Alaska2001
2       Arizona2001
3      Arkansas2001
4    California2001
dtype: object

In [0]:
# Combine identifier with new population column
# Build Single Column for final frame

new = []
final = []
for i in range(2001,2011):
    pop = population0[f'{i}'].to_list()
    new.append(pop)
for i in range(2011,2018):
    pop1 = population1[f'{i}'].to_list()
    new.append(pop1)

for sublist in new:
    for item in sublist:
        final.append(item)



population_column = pd.Series(final)
objects = (data,population_column)
data = pd.concat(objects,axis=1)
data.columns = ['state','population_estimate']
print(data.shape)
data.tail()

(850, 2)


,state,population_estimate
845,Virginia2017,8463587.0
846,Washington2017,7423362.0
847,West Virginia2017,1817004.0
848,Wisconsin2017,5790186.0
849,Wyoming2017,578931.0


In [0]:
print(gdp_sector_data.shape)
gdp_sector_data.head(60)

(107988, 26)


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,"""00000""",United States *,,CAGDP2,1.0,...,All industry total,Thousands of dollars,10581822000,10936418000,11458246000,12213730000,13036637000,13814609000,14451860000,14712845000,14448932000,14992052000,15542582000,16197007000,16784851000,17527258000,18224780000,18715040000,19519424000,20580223000
1,"""00000""",United States *,,CAGDP2,2.0,...,Private industries,Thousands of dollars,9188887407,9462019658,9905898777,10582458507,11326368895,12022614053,12564794482,12731236504,12403879944,12884088643,13405519970,14037519000,14572341002,15255889000,15883937000,16326092000,17065802000,18035586000
2,"""00000""",United States *,,CAGDP2,3.0,11,"Agriculture, forestry, fishing and hunting",Thousands of dollars,99835618,95628974,113953500,142945219,128346703,125130352,144062341,147243983,129967839,146299048,180944850,179573000,215600643,201003000,180655000,164281000,174579000,166464000
3,"""00000""",United States *,,CAGDP2,6.0,21,"Mining, quarrying, and oil and gas extraction",Thousands of dollars,123924346,112417015,138999819,166517463,225677474,273321649,313962886,392174617,275840872,305838102,356315451,358798000,386518621,416375000,259863000,215621000,287324000,346645000
4,"""00000""",United States *,,CAGDP2,10.0,22,Utilities,Thousands of dollars,181332474,177617765,184029170,199216500,198142868,226848389,231906987,241686344,258241011,278837008,287459067,279651000,286340248,298076000,299214000,302355000,315114000,325852000
5,"""00000""",United States *,,CAGDP2,11.0,23,Construction,Thousands of dollars,486484509,493559515,525240363,584642803,651816163,697082819,715327244,648879438,565646742,525125766,524431186,553419000,587552530,636899000,695598000,745534000,790367000,839143000
6,"""00000""",United States *,,CAGDP2,12.0,31-33,Manufacturing,Thousands of dollars,1473849713,1468491776,1524213067,1608098059,1693441358,1793844215,1844745020,1800843857,1702135406,1797010248,1867565573,1927057000,1991904251,2050232000,2126536000,2101178000,2185094000,2321192000
7,"""00000""",United States *,,CAGDP2,13.0,"321,327-339",Durable goods manufacturing,Thousands of dollars,833390176,832841637,863190512,905121072,956799293,1004410039,1030624381,999725846,880960706,964323862,1015190447,1061711000,1101963233,1134092000,1184000000,1190531000,1230670000,1296405000
8,"""00000""",United States *,,CAGDP2,25.0,"311-316,322-326",Nondurable goods manufacturing,Thousands of dollars,640459537,635650138,661022555,702976988,736642065,789434176,814120640,801118011,821174700,832686386,852375126,865346000,889941017,916140000,942537000,910646000,954424000,1024786000
9,"""00000""",United States *,,CAGDP2,34.0,42,Wholesale trade,Thousands of dollars,613751789,613146665,641449945,697110413,754948732,811473488,857827027,884319332,834180413,888905178,934906601,997353000,1040067218,1088175000,1142481000,1133764000,1164604000,1212229000


In [0]:
# Get list of new column names
gdp_columns = gdp_sector_data['Description'][0:34].to_list()

In [0]:
gdp_columns[2]

'  Agriculture, forestry, fishing and hunting'

In [0]:
# Extract State Totals from All areas dataframe
# Concatenate 
concats = []
for state in state_li:
    abb = gdp_sector_data[gdp_sector_data['GeoName']==state]
    concats.append(abb)
    
temp = pd.concat(concats)
temp = temp.reset_index()
gdp_set = temp.copy()

# Drop uneccessary columns

gdp_set = gdp_set.drop(columns=['index','GeoFIPS','Region','TableName',
                                'LineCode','IndustryClassification','Unit'])

gdp = gdp_set.copy()

In [0]:
print(gdp.shape)
gdp.head()

(1700, 20)


,GeoName,Description,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Alabama,All industry total,122449316,127792310,133739237,146524706,155970398,163853599,169678352,172686846,168348037,174753001,181349789,186298987,191481016,194211018,200402713,203829840,210364397,221735502
1,Alabama,Private industries,102670724,106583943,111214744,122795708,130877334,137644699,141320725,143302183,137476539,143259039,149297054,154280965,159253294,161155693,166365821,169210642,174895867,185175681
2,Alabama,"Agriculture, forestry, fishing and hunting",1988838,1588009,2175016,2925346,2630361,2099708,1944306,1894946,1915963,1928347,1591042,1802169,3113696,2518678,2561122,1978866,2308418,2275253
3,Alabama,"Mining, quarrying, and oil and gas extraction",1379713,1207862,1588205,1960549,2747046,2895314,3431980,3849038,2819777,3034470,2949460,2891824,2760597,2524874,2417132,1724871,2783888,3007303
4,Alabama,Utilities,3572672,3442155,3622877,3916215,3824010,4378835,4574488,4719355,5052492,5583912,5820817,5689355,5680349,5860950,6002772,6200305,6154428,6490588


# Transforming Dataframe into new frame layout with the same information:

In [0]:
# find my groups of state gdp information to transform into an observation by the year
print(len(gdp),'rows, divided by 34 rows for each state:',len(gdp)%34)



1700 rows, divided by 34 rows for each state: 0


In [0]:
li = gdp['Description'][0:34].to_list()
obj = []
for i in range(0,1700,34):
    for j in range(2,19):
        df = gdp.iloc[i:i+34,j:j+1]
        obj.append(df)

container = pd.DataFrame(columns=li) 

for i in range(0,17):
    for j in range(0+i,len(obj),17):
        state = obj[j].T
        state.columns = li
        container = container.append(state)
    
    
    

final_gdp = container.reset_index(drop=True)

In [0]:
print(final_gdp.shape)
print(data.shape)
data

(850, 34)
(850, 2)


,state,population_estimate
0,Alabama2001,4480089.0
1,Alaska2001,642337.0
2,Arizona2001,5396255.0
3,Arkansas2001,2705927.0
4,California2001,34871843.0
...,...,...
845,Virginia2017,8463587.0
846,Washington2017,7423362.0
847,West Virginia2017,1817004.0
848,Wisconsin2017,5790186.0


In [0]:
objects = (data,final_gdp)
dframe = pd.concat(objects,axis=1)
print(dframe.shape)

(850, 36)


# Checkpoint:

In [0]:
finalest_frame = dframe.copy()

In [0]:
finalest_frame

,state,population_estimate,All industry total,Private industries,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Durable goods manufacturing,Nondurable goods manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Finance and insurance,Real estate and rental and leasing,Professional and business services,"Professional, scientific, and technical services",Management of companies and enterprises,Administrative and support and waste management and remediation services,"Educational services, health care, and social assistance",Educational services,Health care and social assistance,"Arts, entertainment, recreation, accommodation, and food services","Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Government and government enterprises,Natural resources and mining,Trade,Transportation and utilities,Manufacturing and information,Private goods-producing industries 2/,Private services-providing industries 3/
0,Alabama2001,4480089.0,122449316,102670724,1988838,1379713,3572672,6199991,21192168,11255058,9937110,7250209,9435489,3233047,4215250,19747667,6914072,12833595,9459550,6189588,681598,2588364,8394939,586164,7808775,3270991,437251,2833740,3330197,19778592,3368551,16685698,6805719,NaN,30760711,71910013
1,Alaska2001,642337.0,28420361,22500511,353602,4037335,412750,1430273,947374,111557,835817,718870,1647894,3696983,764578,3555408,875412,2679996,1772032,986375,243387,542270,1575046,109976,1465071,986237,124731,861506,602128,5919850,4390937,2366764,4109733,NaN,6768584,15731927
2,Arizona2001,5396255.0,171474249,148676367,1518311,1286682,3507169,10926930,22223337,19303002,2920335,10041640,13716889,4428573,5600262,36608914,12841216,23767699,17135705,8514266,1684934,6936505,10841516,936880,9904636,7048067,1649312,5398755,3792374,22797883,2804993,23758529,7935741,NaN,35955259,112721108
3,Arkansas2001,2705927.0,70601352,61483503,2321139,409977,1836222,3311111,13887564,7023714,6863850,4439486,5204837,3378595,2563769,9738432,3075485,6662946,5385034,2144896,1869578,1370559,5250846,293447,4957398,1899181,234349,1664832,1857311,9117849,2731116,9644322,5214818,NaN,19929790,41553712
4,California2001,34871843.0,1384111202,1206974416,17706583,6427793,14641450,58705374,177808856,112985396,64823459,76191751,99002817,33587415,92966823,278252558,73288377,204964181,181772130,112885048,27984480,40902601,78906127,10753721,68152407,55403012,17791169,37611843,35601726,177136786,24134376,175194568,48228865,NaN,260648606,946325810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,Virginia2017,8463587.0,509372781,414966890,1719897,1466151,7522893,20268655,43169425,17775531,25393894,21714397,25797653,13592482,17061774,97672053,26144653,71527400,97214162,66501725,12418602,18293834,37863599,5068241,32795359,17208629,3422196,13786433,12695120,94405890,3186048,47512050,21115375,NaN,66624128,348342762
846,Washington2017,7423362.0,524814612,456518728,7656350,482080,4431510,22205763,59974985,45608754,14366231,27278000,43334304,14849826,61870738,92491393,19548036,72943358,54843030,33811646,6727701,14303683,36653209,3124029,33529180,20099009,4232793,15866217,10348531,68295884,8138430,70612303,19281336,NaN,90319178,366199551
847,West Virginia2017,1817004.0,73162915,61389797,313967,7585613,2229668,3384430,7583345,3325437,4257909,3419745,5024252,2220728,1416005,9761262,2557418,7203845,5869047,3267054,704366,1897627,8271719,427847,7843872,2832564,395282,2437282,1477453,11773118,7899580,8443996,4450396,NaN,18867356,42522442
848,Wisconsin2017,5790186.0,321987743,286717491,4648503,1343442,5579614,13160387,60044175,34923330,25120845,19941088,17931194,9588662,11266017,63095124,25589226,37505898,31280489,14199979,9212180,7868331,31855635,3

# Create Energy Consumption columns for a Target(s)

In [0]:
# Get matching years of observation to previous sets
energy1 = energy_data[energy_data['YEAR']>=2002]
print(energy1.shape)



(13301, 5)


In [0]:
# rename columns to simplest form
energy1.columns = ['yr','state','producer','source','consumption2002']

# Extract Target of Natural Gas Consumption from all Industries
energy_nat_gas = energy1[energy1['source']=='Natural Gas (Mcf)']
nat_gas_consumption = energy_nat_gas[energy_nat_gas['producer']=='Total Electric Power Industry']

# Format strings and reset index for better relation
nat_gas_consumption = nat_gas_consumption.apply(lambda x: x.astype(str).str.lower())
nat_gas_consumption = nat_gas_consumption.reset_index()

print(nat_gas_consumption.shape)
nat_gas_consumption.head()



(861, 6)


,index,yr,state,producer,source,consumption2002
0,8429,2002,ak,total electric power industry,natural gas (mcf),41364632
1,8442,2002,al,total electric power industry,natural gas (mcf),131531462
2,8459,2002,ar,total electric power industry,natural gas (mcf),42744550
3,8472,2002,az,total electric power industry,natural gas (mcf),145302961
4,8485,2002,ca,total electric power industry,natural gas (mcf),776884309


In [0]:
nat_gas_consumption['yr'].value_counts()

2011    52
2016    51
2018    51
2002    51
2013    51
2003    51
2015    51
2012    51
2017    51
2014    51
2006    50
2007    50
2004    50
2009    50
2010    50
2008    50
2005    50
Name: yr, dtype: int64

In [0]:
# need 17 values for each state 
# 13 extra rows = (+9 dc - 17 us total - 3 extra from hi) = -13rows

# drop us total values index = (50,101,151,201,251,301,351,401,451,496,501,547,598,649,700,751,802,853)
nat_gas_consumption = nat_gas_consumption.drop([50,101,151,201,251,301,351,401,
                                                451,496,547,598,649,700,751,
                                                802,853])

# drop HI index =(10,61,463)(refer to begining of code)
nat_gas_consumption = nat_gas_consumption.drop([10,61,463])
# 
# add na values to DC (8,59,110,160,210,260,310,360,410)
ins02 = [8,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins03 = [59,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins04 = [110,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins05 = [160,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins06 = [210,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins07 = [260,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins08 = [310,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins09 = [360,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]
ins10 = [410,'District of Columbia2010','total electric power industry','natural gas (mcf)',np.NaN]

In [0]:
nat_gas_consumption['yr'].value_counts()

2016    50
2014    50
2015    50
2013    50
2011    50
2012    50
2017    50
2018    50
2006    49
2004    49
2007    49
2009    49
2010    49
2008    49
2003    49
2002    49
2005    49
Name: yr, dtype: int64

In [0]:
# Build State Abbreviation Dict

state_abv_dict = {'al':'Alabama','ak':'Alaska','az':'Arizona','ar':'Arkansas',
                  'ca':'California','co':'Colorado','ct':'Connecticut','de':'Delaware',
                  'dc':'District of Columbia','fl':'Florida','ga':'Georgia','hi':'Hawaii','id':'Idaho',
                  'il':'Illinois','in':'Indiana','ia':'Iowa','ks':'Kansas','ky':'Kentucky',
                  'la':'Louisiana','me':'Maine','md':'Maryland','ma':'Massachusetts',
                  'mi':'Michigan','mn':'Minnesota','ms':'Mississippi','mo':'Missouri',
                  'mt':'Montana','ne':'Nebraska','nv':'Nevada','nh':'New Hampshire',
                  'nj':'New Jersey','nm':'New Mexico','ny':'New York','nc':'North Carolina',
                  'nd':'North Dakota','oh':'Ohio','ok':'Oklahoma','or':'Oregon','pa':'Pennsylvania',
                  'ri':'Rhode Island','sc':'South Carolina','sd':'South Dakota','tn':'Tennessee',
                  'tx':'Texas','ut':'Utah','vt':'Vermont','va':'Virginia','wa':'Washington',
                  'wv':'West Virginia','wi':'Wisconsin','wy':'Wyoming'}

In [0]:
nat_gas_consumption.head()

,index,yr,state,producer,source,consumption2002
0,8429,2002,ak,total electric power industry,natural gas (mcf),41364632
1,8442,2002,al,total electric power industry,natural gas (mcf),131531462
2,8459,2002,ar,total electric power industry,natural gas (mcf),42744550
3,8472,2002,az,total electric power industry,natural gas (mcf),145302961
4,8485,2002,ca,total electric power industry,natural gas (mcf),776884309


In [0]:
nat_gas_consumption['yr'].astype(int)-1

0      2001
1      2001
2      2001
3      2001
4      2001
       ... 
856    2017
857    2017
858    2017
859    2017
860    2017
Name: yr, Length: 841, dtype: int64

In [0]:
# Replace with Dict
nat_gas_consumption['state']= nat_gas_consumption['state'].replace(state_abv_dict)

# Bond name and state for identifier
prev_year = nat_gas_consumption['yr'].astype(int)-1
nat_gas_consumption['year_'] = prev_year
nat_gas_consumption['state'] = nat_gas_consumption['state']+nat_gas_consumption['year_'].astype(str)

# Drop The year column after checking random spots
nat_gas_consumption = nat_gas_consumption.drop(columns=['yr','year_'])

print(nat_gas_consumption.shape)
nat_gas_consumption.head()

(841, 5)


,index,state,producer,source,consumption2002
0,8429,Alaska2001,total electric power industry,natural gas (mcf),41364632
1,8442,Alabama2001,total electric power industry,natural gas (mcf),131531462
2,8459,Arkansas2001,total electric power industry,natural gas (mcf),42744550
3,8472,Arizona2001,total electric power industry,natural gas (mcf),145302961
4,8485,California2001,total electric power industry,natural gas (mcf),776884309


In [0]:

def Insert_row(row_number, df, row_value): 
    # Starting value of upper half 
    start_upper = 0
   
    # End value of upper half 
    end_upper = row_number 
   
    # Start value of lower half 
    start_lower = row_number 
   
    # End value of lower half 
    end_lower = df.shape[0] 
   
    # Create a list of upper_half index 
    upper_half = [*range(start_upper, end_upper, 1)] 
   
    # Create a list of lower_half index 
    lower_half = [*range(start_lower, end_lower, 1)] 
   
    # Increment the value of lower half by 1 
    lower_half = [x.__add__(1) for x in lower_half] 
   
    # Combine the two lists 
    index_ = upper_half + lower_half 
   
    # Update the index of the dataframe 
    df.index = index_ 
   
    # Insert a row at the end 
    df.loc[row_number] = row_value 
    
    # Sort the index labels 
    df = df.sort_index() 
   
    # return the dataframe 
    return df 



In [0]:

# add na values to DC (8,59,110,160,210,260,310,360,410)

row_number = 8
row_value = ins02  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)


row_number = 59
row_value = ins03  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

row_number = 110
row_value = ins04  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

row_number = 160
row_value = ins05  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)


row_number = 210
row_value = ins06  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

row_number = 310
row_value = ins07  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

row_number = 360
row_value = ins08  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

row_number = 410
row_value = ins09  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

row_number = 460
row_value = ins10  
  
if row_number > nat_gas_consumption.index.max()+1: 
    print("Invalid row_number") 
else: 
      
    # Let's call the function and insert the row 
    # at the second position 
    nat_gas_consumption = Insert_row(row_number, nat_gas_consumption, row_value) 
   


print(nat_gas_consumption.shape)

(842, 5)
(843, 5)
(844, 5)
(845, 5)
(846, 5)
(847, 5)
(848, 5)
(849, 5)
(850, 5)


In [0]:
# Drop uneccessary columns
consumption_final = nat_gas_consumption.drop(columns=['index','producer','source'])
# Rename remaining columns
consumption_final.columns = ['state','total natural gas consumption for electric power2002']

# ind = 1
# consumption_final.insert(loc=ind,column='year_of_observation',value = year_index)

print(consumption_final.shape)
consumption_final.head()

(850, 2)


,state,total natural gas consumption for electric power2002
0,Alaska2001,41364632
1,Alabama2001,131531462
2,Arkansas2001,42744550
3,Arizona2001,145302961
4,California2001,776884309


# Combine target with features

In [0]:
print(finalest_frame.shape,consumption_final.shape)

finalest_frame.tail()

(850, 36) (850, 2)


,state,population_estimate,All industry total,Private industries,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Durable goods manufacturing,Nondurable goods manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Finance and insurance,Real estate and rental and leasing,Professional and business services,"Professional, scientific, and technical services",Management of companies and enterprises,Administrative and support and waste management and remediation services,"Educational services, health care, and social assistance",Educational services,Health care and social assistance,"Arts, entertainment, recreation, accommodation, and food services","Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Government and government enterprises,Natural resources and mining,Trade,Transportation and utilities,Manufacturing and information,Private goods-producing industries 2/,Private services-providing industries 3/
845,Virginia2017,8463587.0,509372781,414966890,1719897,1466151,7522893,20268655,43169425,17775531,25393894,21714397,25797653,13592482,17061774,97672053,26144653,71527400,97214162,66501725,12418602,18293834,37863599,5068241,32795359,17208629,3422196,13786433,12695120,94405890,3186048,47512050,21115375,NaN,66624128,348342762
846,Washington2017,7423362.0,524814612,456518728,7656350,482080,4431510,22205763,59974985,45608754,14366231,27278000,43334304,14849826,61870738,92491393,19548036,72943358,54843030,33811646,6727701,14303683,36653209,3124029,33529180,20099009,4232793,15866217,10348531,68295884,8138430,70612303,19281336,NaN,90319178,366199551
847,West Virginia2017,1817004.0,73162915,61389797,313967,7585613,2229668,3384430,7583345,3325437,4257909,3419745,5024252,2220728,1416005,9761262,2557418,7203845,5869047,3267054,704366,1897627,8271719,427847,7843872,2832564,395282,2437282,1477453,11773118,7899580,8443996,4450396,NaN,18867356,42522442
848,Wisconsin2017,5790186.0,321987743,286717491,4648503,1343442,5579614,13160387,60044175,34923330,25120845,19941088,17931194,9588662,11266017,63095124,25589226,37505898,31280489,14199979,9212180,7868331,31855635,3541793,28313842,10058911,2355135,7703776,6924251,35270252,5991945,37872281,15168276,NaN,79196507,207520984
849,Wyoming2017,578931.0,37453599,31246837,639449,7373433,1051762,1993913,1946179,475691,1470488,1473687,2104193,3187562,631307,5053187,1005872,4047315,1796622,1129261,121464,545897,1780050,114394,1665656,1628767,273483,1355284,586725,6206762,8012882,3577880,4239325,NaN,11952974,19293863


In [0]:
FinalFrame = pd.merge(finalest_frame,consumption_final)

FinalFrame

,state,population_estimate,All industry total,Private industries,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Durable goods manufacturing,Nondurable goods manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Finance and insurance,Real estate and rental and leasing,Professional and business services,"Professional, scientific, and technical services",Management of companies and enterprises,Administrative and support and waste management and remediation services,"Educational services, health care, and social assistance",Educational services,Health care and social assistance,"Arts, entertainment, recreation, accommodation, and food services","Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Government and government enterprises,Natural resources and mining,Trade,Transportation and utilities,Manufacturing and information,Private goods-producing industries 2/,Private services-providing industries 3/,total natural gas consumption for electric power2002
0,Alabama2001,4480089.0,122449316,102670724,1988838,1379713,3572672,6199991,21192168,11255058,9937110,7250209,9435489,3233047,4215250,19747667,6914072,12833595,9459550,6189588,681598,2588364,8394939,586164,7808775,3270991,437251,2833740,3330197,19778592,3368551,16685698,6805719,NaN,30760711,71910013,131531462
1,Alaska2001,642337.0,28420361,22500511,353602,4037335,412750,1430273,947374,111557,835817,718870,1647894,3696983,764578,3555408,875412,2679996,1772032,986375,243387,542270,1575046,109976,1465071,986237,124731,861506,602128,5919850,4390937,2366764,4109733,NaN,6768584,15731927,41364632
2,Arizona2001,5396255.0,171474249,148676367,1518311,1286682,3507169,10926930,22223337,19303002,2920335,10041640,13716889,4428573,5600262,36608914,12841216,23767699,17135705,8514266,1684934,6936505,10841516,936880,9904636,7048067,1649312,5398755,3792374,22797883,2804993,23758529,7935741,NaN,35955259,112721108,145302961
3,Arkansas2001,2705927.0,70601352,61483503,2321139,409977,1836222,3311111,13887564,7023714,6863850,4439486,5204837,3378595,2563769,9738432,3075485,6662946,5385034,2144896,1869578,1370559,5250846,293447,4957398,1899181,234349,1664832,1857311,9117849,2731116,9644322,5214818,NaN,19929790,41553712,42744550
4,California2001,34871843.0,1384111202,1206974416,17706583,6427793,14641450,58705374,177808856,112985396,64823459,76191751,99002817,33587415,92966823,278252558,73288377,204964181,181772130,112885048,27984480,40902601,78906127,10753721,68152407,55403012,17791169,37611843,35601726,177136786,24134376,175194568,48228865,NaN,260648606,946325810,776884309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,Virginia2017,8463587.0,509372781,414966890,1719897,1466151,7522893,20268655,43169425,17775531,25393894,21714397,25797653,13592482,17061774,97672053,26144653,71527400,97214162,66501725,12418602,18293834,37863599,5068241,32795359,17208629,3422196,13786433,12695120,94405890,3186048,47512050,21115375,NaN,66624128,348342762,364054581
846,Washington2017,7423362.0,524814612,456518728,7656350,482080,4431510,22205763,59974985,45608754,14366231,27278000,43334304,14849826,61870738,92491393,19548036,72943358,54843030,33811646,6727701,14303683,36653209,3124029,33529180,20099009,4232793,15866217,10348531,68295884,8138430,70612303,19281336,NaN,90319178,366199551,73385589
847,West Virginia2017,1817004.0,73162915,61389797,313967,7585613,2229668,3384430,7583345,3325437,4257909,3419745,5024252,2220728,1416005,9761262,2557418,7203845,5869047,3267054,704366,1897627,8271719,427847,7843872,2832564,395282,2437282,1477453,11773118,7899580,8443996,4450396,NaN,18867356,42522442,12112462
848,Wisconsin2017,5790186.0,321987743,286717491,4648503,1343442,5579614,13160387,

In [0]:
gdp_energy['state'] = gdp_energy['state'].astype('string')
gdp_energy = gdp_energy.replace('(D)',np.NaN)
obj_columns = gdp_energy.select_dtypes(include='object').columns.to_list()

gdp_energy_nums = gdp_energy[obj_columns].astype(float)
gdp_energy_millions = gdp_energy_nums/1000000
gdp_energy_millions = gdp_energy_millions.round(3)

gdp_energy_population = gdp_energy['population_estimate']/1000000

obs = (gdp_energy['state'],gdp_energy_population.round(3),gdp_energy_millions)
gdp_energy_final=pd.concat(obs,axis=1)

gdp_energy_final = gdp_energy_final.dropna()

# CSV DOWNLOAD

In [0]:
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink, FileLinks
    display(FileLink(csv_file_name))
# Diplay download link:
csv_download_link(FinalFrame, 'Project.csv')

/Users/jp/DS-Unit-2-Applied-Modeling/module1-define-ml-problems/Project.csv